In [2]:
from faker import Faker
import csv
import random
from datetime import datetime, timedelta
import string

# 建立 Faker 實例
fake = Faker()

# 建立一個列表來儲存所有的 UserID
user_ids = [fake.unique.random_number(digits=5) for _ in range(10000)]

# 為 event_master.USER 表格生成資料
with open('users.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["UserID", "Name", "Account", "Password", "Profile_pid_url"])
    for UserID in user_ids:
        Name = fake.name()
        Account = fake.unique.email()
        Password = fake.password(length=10)
        Profile_pid_url = fake.image_url()
        writer.writerow([UserID, Name, Account, Password, Profile_pid_url])
# 隨機選擇 10 個 UserID
admin_ids = random.sample(user_ids, 10)

# 將這些 UserID 寫入 isAdmin.csv
with open('isAdmin.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["UserID"])  # 寫入標題行
    for admin_id in admin_ids:
        writer.writerow([admin_id])
        
# 為 event_master.PRIVATE_EVENT 表格生成資料
with open('private_event.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EventID", "UserID", "Event_Start", "Event_End", "Name", "Description"])
    for UserID in user_ids:
        num_events = random.randint(1, 10)  # 每個使用者有 0 到 10 個事件
        for _ in range(num_events):
            EventID = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
            Event_Start = fake.date_time_between(start_date='-1y', end_date='+1y')
            Event_End = Event_Start + timedelta(hours=random.randint(1, 24))
            Name = fake.text(max_nb_chars=10)  # Name 的長度不超過 10 個字元
            Description = fake.text()
            writer.writerow([EventID, UserID, Event_Start, Event_End, Name, Description])

# 建立一個列表來儲存所有的 GroupID
group_ids = [i for i in range(1, 401)]

# 為 event_master.GROUP 表格生成資料
with open('group.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["GroupID", "Name"])
    for GroupID in group_ids:
        Name = fake.word()
        writer.writerow([GroupID, Name])

# 建立一個字典來儲存每個群組的成員
group_members = {GroupID: random.sample(user_ids, random.randint(1, 100)) for GroupID in group_ids}

# 為 event_master.GROUP_HAS_USER 表格生成資料
with open('group_has_user.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["GroupID", "UserID"])
    for GroupID, UserIDs in group_members.items():
        for UserID in UserIDs:
            writer.writerow([GroupID, UserID])

# 為 event_master.GROUP_HAS_MANAGER 表格生成資料
with open('group_has_manager.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["GroupID", "UserID"])
    for GroupID in group_ids:
        UserID = random.choice(user_ids)
        writer.writerow([GroupID, UserID])

# 建立一個列表來儲存所有的 EventID
event_ids = [fake.unique.random_number(digits=10) for _ in range(1000)]

# 建立一個列表來儲存所有的 Status
status_list = ['In_Voting', 'End_Voting', 'Not_Start_Yet', 'On_Going', 'Closure']

# 建立一個字典來儲存每個群組的所有事件
group_events = {GroupID: [] for GroupID in group_ids}
# 建立一個字典來儲存每個群組事件的投票時間
group_event_vote_times = {}

# 為 event_master.GROUP_EVENT 表格生成資料
with open('group_event.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EventID", "GroupID", "Name", "Description", "Event_Start", "Event_End", "Status", "OrganizerID", "Vote_Start", "Vote_End", "VoteDeadline", "HavePossibility"])
    for EventID in event_ids:
        GroupID = random.choice(group_ids)
        group_events[GroupID].append(EventID)
        Name = fake.word()
        Description = fake.text()
        Status = random.choice(status_list)
        OrganizerID = random.choice(group_members[GroupID])  # OrganizerID 是該群組的成員
        Vote_Start = fake.date_time_between(start_date='-1y', end_date='+1y')
        Vote_End = Vote_Start + timedelta(days=random.randint(1, 10))  # 投票時間段的結束時間
        group_event_vote_times[EventID] = (Vote_Start, Vote_End)  # 將投票時間添加到字典中
        VoteDeadline = Vote_Start - timedelta(days=1)  # 投票的截止時間
        HavePossibility = random.choice([True, False])
        # 在投票結束時才會決定 Event_Start 和 Event_End
        if Status in ['Not_Start_Yet', 'On_Going', 'Closure']:
            Event_Start = Vote_End + timedelta(days=random.randint(1, 10))
            Event_End = Event_Start + timedelta(days=random.randint(1, 10))
        else:
            Event_Start = None
            Event_End = None
        writer.writerow([EventID, GroupID, Name, Description, Event_Start, Event_End, Status, OrganizerID, Vote_Start, Vote_End, VoteDeadline, HavePossibility])

# 為 event_master.AVAILABLE_TIME 表格生成資料
with open('available_time.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["UserID", "EventID", "Available_Start", "Possibility_level"])
    for GroupID in group_ids:
        for UserID in group_members[GroupID]:  # 每一位成員都要決定他的可用時間
            for EventID in group_events[GroupID]:  # 每一個群組事件
                # 在投票的時間範圍內隨機選擇多個時間作為可用時間的開始時間
                Vote_Start, Vote_End = group_event_vote_times[EventID]
                num_intervals = int((Vote_End - Vote_Start).total_seconds() / 1800)
                for i in range(num_intervals):
                    Available_Start = Vote_Start + timedelta(minutes=30*i)
                    # 隨機決定可能性等級
                    Possibility_level = random.choice(['Definitely', 'Maybe'])
                    writer.writerow([UserID, EventID, Available_Start, Possibility_level])

# 為 event_master.USER_JOIN_EVENT 表格生成資料
with open('user_join_event.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["UserID", "EventID", "IsAccepted"])
    for GroupID in group_ids:
        for UserID in group_members[GroupID]:  # 每一位成員都要決定他是否參加
            for EventID in group_events[GroupID]:  # 每一個群組事件
                # 隨機決定該使用者是否接受參加該事件
                IsAccepted = random.choice([True, False])
                writer.writerow([UserID, EventID, IsAccepted])

# 為 event_master.TODO 表格生成資料
with open('todo.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["TodoID", "GroupID", "AssigneeID", "AssignerID", "Name", "Description", "Completed", "Deadline"])
    todo_id = 1
    for GroupID in group_ids:
        for UserID in group_members[GroupID]:  # 每一位成員都要生成待辦事項
            for _ in range(random.randint(0, 10)):  # 每個人生成0~10筆待辦事項
                AssigneeID = UserID
                AssignerID = random.choice(group_members[GroupID])  # 待辦事項的指派者是該群組的成員
                Name = fake.word()
                Description = fake.text()
                Completed = random.choice([True, False])
                Deadline = fake.date_time_between(start_date='+1d', end_date='+1y')
                writer.writerow([str(todo_id).zfill(10), GroupID, AssigneeID, AssignerID, Name, Description, Completed, Deadline])
                todo_id += 1

# 為 event_master.CHAT 表格生成資料
with open('chat.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["GroupID", "SpeakerID", "Timing", "Content"])
    for GroupID, UserIDs in group_members.items():
        for UserID in UserIDs:
            for _ in range(random.randint(0, 10)):
                Timing = fake.date_time()
                Content = fake.text()
                writer.writerow([GroupID, UserID, Timing, Content])